# Sentinel-2 Sunglint Finder with Earth Engine Python API

This notebook shows how to find Sentinel-2 scenes over sunglint geometry using the Earth Engine Python API.  
Adjust the `bbox`, `start_date`, `end_date`, and other parameters as needed.


In [1]:
# 1. Install and import Earth Engine Python API
# Uncomment the following line if running in Colab or a new environment:
# !pip install earthengine-api

import ee
import math

# Initialize the Earth Engine client. 
# If running for the first time, you may need to authenticate:
# ee.Authenticate()
ee.Initialize()
print('Earth Engine initialized.')


Earth Engine initialized.


In [2]:
# 2. Define parameters: bounding box, date range, sunglint threshold, and cloud filter

# -------------------------------------------------------------------
# Modify these variables as needed:
xmin, ymin, xmax, ymax = -80.0, 20.0, -75.0, 25.0  # Example bbox [lon_min, lat_min, lon_max, lat_max]
start_date = '2021-06-01'
end_date   = '2021-08-31'

sun_glint_threshold = 30  # Degrees; scenes with glint angle ≤ this will be selected.
max_cloud_cover = 20      # Maximum cloud percentage for initial filter (0-100)
# -------------------------------------------------------------------

# Create an Earth Engine geometry for the bounding box
bbox = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])


In [8]:
# 3. Load Sentinel-2 L1C collection, filter by date, bounds, and cloud cover

s2_collection = (ee.ImageCollection('COPERNICUS/S2') \
                .filterDate('2023-01-01', '2023-01-31') \
                .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)))

print('Total S2 scenes in date/area filter:', s2_collection.size().getInfo())


s2_glint_scenes = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2023-01-01', '2023-01-31') \
    .filter(ee.Filter.gt('MEAN_SOLAR_ZENITH_ANGLE', 60)) \
    .filter(ee.Filter.lt('MEAN_INCIDENCE_ZENITH_ANGLE_B8', 10)) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))

print('Total sunglint scenes in date/area filter:',s2_glint_scenes.size().getInfo())

Total S2 scenes in date/area filter: 107288
Total sunglint scenes in date/area filter: 22742


In [9]:
# 6. (Optional) Mask out non-water pixels using JRC Surface Water dataset
# Load JRC Global Surface Water Occurrence band (0-100%)
jrc = ee.Image('JRC/GSW1_2/GlobalSurfaceWater').select('occurrence')
water_mask = jrc.gt(50)  # Keep pixels that are water >50% of the year

def mask_non_water(img):
    return img.updateMask(water_mask)

s2_glint_water = s2_glint_scenes.map(mask_non_water)
print('Applied water mask: resulting scenes:', s2_glint_water.size().getInfo())


Applied water mask: resulting scenes: 22742


In [15]:
import geemap

# Define visualization parameters if not already defined
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.2
}

# If Map is not defined, import geemap and create a map
Map = geemap.Map(center=[(ymin + ymax) / 2, (xmin + xmax) / 2], zoom=8)

# Get the first image from the s2_glint_water collection
first_glint_water = s2_glint_water.first()

# Add the first scene to the map with visualization parameters
Map.addLayer(first_glint_water, vis_params, 'First S2 Sunglint Water Scene')
Map

Map(center=[22.5, -77.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…